In [21]:
from pyforest import *
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import quickda
from quickda.explore_data import * 
from quickda.explore_categoric import * 
from quickda.explore_numeric import *
from quickda.explore_numeric_categoric import * 
from quickda.clean_data import *
from statsmodels import *

## Working with combined August Data

In [2]:
# reading in data and converting date to time
aug_data = pd.read_csv("/Users/lukehoward/Desktop/RMIT_SEM_2_2021/Case Studies/Project/Data/merged_data_aug_trim.csv")
aug_data.date = pd.to_datetime(aug_data.date, infer_datetime_format=True, dayfirst=True )


In [3]:
# filtering data for chosen states and time frame (removing negative daily doses also)
aug_data.query("state == 'NSW' or state == 'VIC' or state == 'QLD'", inplace=True)
aug_data.query("date <= '2021-08-31'", inplace =True)
aug_data.query("daily_doses > 0", inplace =True)

In [4]:
#back filling missing values
aug_data.bfill(axis = 0, inplace = True)

In [26]:
# Create figure with secondary y-axis
aug_data_qld = aug_data[aug_data["state"] == "QLD"]

fig = px.scatter(aug_data_qld, x = "transcript_sentiment_positive", y = "avr_positive_tweet_sentiment", trendline="ols")
fig.update_xaxes(title = "Transcript Sentiment")
fig.update_yaxes(title = "Twitter Sentiment")
fig.show()

In [6]:
aug_data.columns

Index(['state', 'date', 'daily_newcase', 'total_doses', 'daily_doses',
       'transcript_sentiment_positive', 'transcript_sentiment_neutral',
       'transcript_sentiment_negative', 'avr_positive_tweet_sentiment',
       'avr_neutral_tweet_sentiment', 'avr_negative_tweet_sentiment'],
      dtype='object')

## Dash Plots

In [7]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from datetime import date

Have included the graph descriptions in dcc.Markdown below. All appear directly the below the graph they correspond to.

In [33]:
# Build App
app = JupyterDash(__name__)
app.layout =  html.Div([
    html.H1("Transcript Sentiment ", style={'textAlign': 'center'}),
    
    html.H2(children='''
        A visualisation of state sentiment through time
        ''', style={'textAlign': 'center'}),
    
    html.Label('State:'),
    
     dcc.Dropdown(
            id = "states_dropdown",
            options = [
                {'label' : "Victoria", "value" : 'VIC'},
                {'label' : "Queensland", "value" : 'QLD'},
                {'label' : "New South Wales", "value" : 'NSW'}],
            value = "VIC"
        ),
    
    html.Label('Transcript Sentiment:'),
    dcc.Dropdown(
            id = "sentiments_dropdown",
            options = [
                {'label' : "Negative", "value" : 'transcript_sentiment_negative'},
                {'label' : "Neutral", "value" : 'transcript_sentiment_neutral'},
                {'label' : "Positive", "value" : 'transcript_sentiment_positive'}],
            value = "transcript_sentiment_negative"
        ),
     
      html.Label('Twitter Sentiment:'),
     dcc.Dropdown(
            id = "twitter_dropdown",
            options = [
                {'label' : "Negative", "value" : 'avr_negative_tweet_sentiment'},
                {'label' : "Neutral", "value" : 'avr_neutral_tweet_sentiment'},
                {'label' : "Positive", "value" : 'avr_positive_tweet_sentiment'}],
            value = "avr_negative_tweet_sentiment"
        ),
   
    
    dcc.Graph(id='sentiment_graph'), 
    
    dcc.Markdown(
        '''
        The above graph shows the change in sentiment from August 1 to August 31 for Vicotria, Queensland and New South Wales. 
        NSW is consistantly more negative than other states, whilst QLD and VIC are more positive. 
        All states show a relatively low proportion of neutral sentiment

        '''
    ),
    
    html.H2(children='''

        A visualisation of state sentiment sentiment vs daily doses
        ''', style={'textAlign': 'center'}),  
    
    dcc.Graph(id='states_graph'), 
    
    dcc.Markdown(
        '''
        The above graph shows the change in goverment press conference sentiment and the daily doses of covid-19 vaccine administered.
        The top portion shows the number of doses in thousands, bottom portion shows the sentiment proportion of transcript for each
        date. 

        '''
    ),
    
    dcc.Graph(id='twitter_transcript'),
    
    dcc.Markdown(
        '''
        The scatterplot allows users to compare transcript and twitter sentiment for selected state. 
        Input is controlled by the dropdowns at the top. 
        Trendline shows the strength of the relationship. 
        Appears as though there is not a strong relationship between transcript and twitter sentiment.
        

        '''
    ),
])

# State sentiment through time graph function and callback
@app.callback(
    Output('sentiment_graph', 'figure'),
    Input('sentiments_dropdown', 'value'))
def update_figure(selected_sentiment):
    fig = px.line(aug_data, x = "date", y =selected_sentiment , color = "state", template = "simple_white")
    fig = fig.update_layout(
                   xaxis_title='Date',
                   yaxis_title='Sentiment Proportion')

    return fig

# Positive sentiment and daily doses graph call back and function
@app.callback(
    Output('states_graph', 'figure'),
    [Input('states_dropdown', 'value'),
     Input('sentiments_dropdown', 'value')])
def update_figure2(selected_state, selected_sentiment):
    filtdf= aug_data[aug_data["state"] == selected_state]
    fig = make_subplots(rows = 2, shared_xaxes = True, vertical_spacing = 0.15)

    # Add traces
    fig.add_trace(
        go.Scatter(x=filtdf["date"], y=filtdf["daily_doses"], name="number of doses"),
        row = 1, col =1
    )

    fig.add_trace(
        go.Bar(x=filtdf["date"], y=filtdf[selected_sentiment], name="proportion sentiment" ),
        row = 2, col = 1
    )
    #  adding y-axis labels
    fig.update_yaxes(title = str.title("number of daily vaccination doses"), row =1, col = 1,
                 title_font = {"size":8},
                 title_standoff = 30)

    fig.update_yaxes(title = str.title("proportion of sentiment"), row =2, col = 1,
                 title_font = {"size":8},
                 title_standoff = 30)

    # Add figure title
    fig.update_layout(
        title_text= str.title("number of doses and sentiment through time"), 
        template = "simple_white"
    )
    return fig
@app.callback(
    Output('twitter_transcript', 'figure'),
    [Input('states_dropdown', 'value'),
     Input('sentiments_dropdown', 'value'),
     Input('twitter_dropdown','value')])
def update_figure(selected_state, trans_sentiment,tweet_sentiment ):
    filt = aug_data[aug_data["state"] == selected_state]

    fig = px.scatter(filt, x = trans_sentiment, y = tweet_sentiment, trendline="ols")
    fig.update_xaxes(title = "Transcript Sentiment")
    fig.update_yaxes(title = "Twitter Sentiment")
    fig.update_layout(
        title_text= str.title("Impact of transcript sentiment of twitter"), 
        template = "simple_white"
    )
    return fig

# Twitter vs Transcript sentiment

# Run app and display result inline in the notebook
app.run_server(debug = True)

Dash app running on http://127.0.0.1:8050/
